In [33]:
import pandas as pd
from sqlalchemy import create_engine
from custom_func import *

In [16]:
LAND = (# коди, що пов'язані з податком на землю
    "18010500",
    "18010600",
    "18010700",
    "18010800",
    "18010900"
)

REGIONS = {
    "02000000000" : "Вінницька",
    "03000000000" : "Волинська",
    "04000000000" : "Дніпропетровська",
    "05000000000" : "Донецька",
    "06000000000" : "Житомирська",
    "07000000000" : "Закарпатська",
    "08000000000" : "Запорізька",
    "09000000000" : "Івано-Франківська",
    "10000000000" : "Київська",
    "11000000000" : "Кіровоградська",
    "12000000000" : "Луганська",
    "13000000000" : "Львівська",
    "14000000000" : "Миколаївська", 
    "15000000000" : "Одеська",
    "16000000000" : "Полтавська",
    "17000000000" : "Рівненська",
    "18000000000" : "Сумська",
    "19000000000" : "Тернопільська",
    "20000000000" : "Харківська",
    "21000000000" : "Херсонська",
    "22000000000" : "Хмельницька",
    "23000000000" : "Черкаська",
    "24000000000" : "Чернівецька",
    "25000000000" : "Чернігівська",
}

In [23]:
def db_connect(query: str, conn: str):
    """ Напряму вантажить дані з бази. """
    db_connection = create_engine(conn)
    return pd.read_sql(query, db_connection)

def generate_query(year):
    
    query = f"""
        SELECT "ADMIN", "FIN_SOURCE", "INCO", "EXECUTED", "DATE"
        FROM "Budget"."dbo_OpenBudgetIncomes"
        WHERE "ADMIN" IN ('02000000000',
                          '03000000000',
                          '04000000000',
                          '05000000000',
                          '06000000000',
                          '07000000000',
                          '08000000000',
                          '09000000000',
                          '10000000000',
                          '11000000000',
                          '12000000000',
                          '13000000000',
                          '14000000000',
                          '15000000000',
                          '16000000000',
                          '17000000000',
                          '18000000000',
                          '19000000000',
                          '20000000000',
                          '21000000000',
                          '22000000000',
                          '23000000000',
                          '24000000000',
                          '25000000000')
          AND "DATE" >= '{year}-01-01'
          AND "DATE" <= '{year}-03-01';"""
    
    return query


def incomes(connection):
    
    current, previous = generate_query(2020), generate_query(2019)
    
    df_current = db_connect(current, connection)
    df_previous = db_connect(previous, connection)
    
    # земля
    land = df_current.loc[df_current["INCO"].isin(LAND)] \
                     .groupby("ADMIN", as_index=False)["EXECUTED"].sum() \
                     .rename(columns={"EXECUTED": "Плата за землю (p2_05)"})
    
    # ПДФО
    pdfo = df_current.loc[df_current["INCO"].eq("11010000")] \
                     .groupby("ADMIN", as_index=False)["EXECUTED"].sum() \
                     .rename(columns={"EXECUTED": "Податок на дохід фіз осіб (для p2_01)"})
    
    # Загальний фонд, без оф. трансферт \ цей рік
    
    mask = df_current["FIN_SOURCE"].eq("C") & df_current["INCO"].str.contains("^[1235]0000000")
    wo_transfers = df_current.loc[mask] \
                             .groupby("ADMIN", as_index=False)["EXECUTED"].sum() \
                             .rename(columns={"EXECUTED": "Дохід без міжбюдж. трансфертів (p2_02)"})
    
    # Загальний фонд, без оф. трансферт \ попередній рік
    mask = df_previous["FIN_SOURCE"].eq("C") & df_previous["INCO"].str.contains("^[1235]0000000")
    wo_transfers_previous = df_previous.loc[mask] \
                                       .groupby("ADMIN", as_index=False)["EXECUTED"].sum() \
                                       .rename(columns={"EXECUTED": "Дохід без міжбюдж. трансфертів _ ПОПЕРЕДНІЙ РІК"})
    
    # agg
    budget_agg = pd.DataFrame(land["ADMIN"])
    for df in (land, pdfo, wo_transfers, wo_transfers_previous):
        budget_agg = budget_agg.merge(df)
    
#     budget_agg.insert(0, "Область", budget_agg["ADMIN"].map(REGIONS))
#     budget_agg["Населення"] = budget_agg["Область"].map(dict_population)
#     budget_agg["Податки на одну особу (p2_01)"]  = budget_agg["Податок на дохід фіз осіб (для p2_01)"] / budget_agg["Населення"]

    return budget_agg

In [6]:
with open("./../psql_engine.txt", "r") as f:
    connection = f.read()

In [27]:
query_p2_4 = f"""
SELECT "ADMIN", SUM("EXECUTED")
FROM "Budget"."dbo_OpenBudgetExpenses"
WHERE "ADMIN" IN ('02000000000',
                  '03000000000',
                  '04000000000',
                  '05000000000',
                  '06000000000',
                  '07000000000',
                  '08000000000',
                  '09000000000',
                  '10000000000',
                  '11000000000',
                  '12000000000',
                  '13000000000',
                  '14000000000',
                  '15000000000',
                  '16000000000',
                  '17000000000',
                  '18000000000',
                  '19000000000',
                  '20000000000',
                  '21000000000',
                  '22000000000',
                  '23000000000',
                  '24000000000',
                  '25000000000')
  AND "ECON" ~ '^3'
  AND "DATE" >= '2020-01-01'
  AND "DATE" <= '2020-03-01'
GROUP BY "ADMIN";"""

df1 = db_connect(query_p2_4, connection)
df1 = df1.rename(columns={"sum": "Капітальні видатки (p2_04)"})
df2 = incomes(connection)

In [30]:
result = pd.merge(df1, df2, on="ADMIN")

In [31]:
result.insert(0, "Область", result["ADMIN"].map(REGIONS))
result["Населення"] = result["Область"].map(dict_population)
result["Податки на одну особу (p2_01)"]  = result["Податок на дохід фіз осіб (для p2_01)"] / result["Населення"]

In [32]:
result

,Область,ADMIN,Капітальні видатки (p2_04),Плата за землю (p2_05),Податок на дохід фіз осіб (для p2_01),Дохід без міжбюдж. трансфертів (p2_02),Дохід без міжбюдж. трансфертів _ ПОПЕРЕДНІЙ РІК,Населення,Податки на одну особу (p2_01)
0,Вінницька,02000000000,8.421927e+07,1.439203e+08,8.648175e+08,1.362296e+09,1.981673e+09,1541782.0,560.920740
1,Волинська,03000000000,1.269372e+08,7.804511e+07,5.365692e+08,8.559776e+08,1.075828e+09,1029736.0,521.074557
2,Дніпропетровська,04000000000,1.646014e+09,7.291244e+08,4.515191e+09,6.955396e+09,6.628021e+09,3180809.0,1419.510257
3,Донецька,05000000000,3.688550e+08,2.430432e+08,2.087742e+09,2.922402e+09,2.896017e+09,4127532.0,505.808859
4,Житомирська,06000000000,1.732043e+08,1.238414e+08,7.110392e+08,1.159912e+09,1.483856e+09,1212157.0,586.590004
5,Закарпатська,07000000000,2.022875e+08,8.975767e+07,6.966198e+08,1.106436e+09,1.121605e+09,1251467.0,556.642596
6,Запорізька,08000000000,2.983307e+08,3.431066e+08,1.836487e+09,2.862062e+09,2.793065e+09,1691424.0,1085.763570
7,Івано-Франківська,09000000000,1.993527e+08,9.334467e+07,5.132831e+08,8.468402e+08,1.342024e+09,1366161.0,375.711991
8,Київська,10000000000,6.407366e+08,3.422171e+08,2.238909e+09,3.750055e+09,3.331908e+09,1771229.0,1264.042566
9,Кіровоградська,11000000000,1.042679e+08,1.917324e+08,8.191793e+08,1.372264e+09,1.224725e+09,930013.0,880.825644
